# Behavior Cloning

### Data Augmentation and Formatting
Flip images and measurements

In [1]:
import csv
import cv2
import numpy as np

In [150]:
from skimage.transform import resize


lines = []
with open("./data/driving_log.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        filename = line[0].split('/')
        try:
            old_filename = '/'.join(['data',filename[-2],filename[-1]])
            new_filename = '/'.join(['data_cropped',filename[-2],filename[-1]])
            img = cv2.imread(old_filename, cv2.IMREAD_COLOR)
            img = resize(img, (32,32), mode="reflect")
            cv2.imwrite(new_filename, img)
            
        except Exception as e:
            print(e)
            print(old_filename, "::", new_filename)
        lines.append([new_filename, *line[-4:]])

print(len(lines))
fw = open("./data_cropped/driving_log_updated.csv", "w")
for line in lines:
    fw.write("{},{},{},{},{}\n".format(*line))
    if None and abs(float(line[1])) > 0.3:
        new_filename = line[0][:-4]+"_flipped.jpg"
        img = cv2.imread(line[0], cv2.IMREAD_COLOR)
        img = np.fliplr(img)
        fw.write("{},{},{},{},{}\n".format(new_filename, -1.0 * float(line[1]), line[2], line[3], line[4]))
        cv2.imwrite(new_filename, img)
    
fw.close()

19001


Crop Images

In [ ]:
# 55 off top
# 30 off bottom
#fw = open("./data/driving_log_updated_cropped.csv", "w")
#with open("./data/driving_log_updated.csv") as f:
#    reader = csv.reader(f)
#    for line in reader:
#        filename = line[0]
#        img = cv2.imread(filename)
#        img = img[55:-30, :]
#        f_split = filename.split("/")
#        f_split[-2] = "CROPPED_IMG"
#        new_filename = '/'.join(f_split)
#        fw.write("{},{},{},{},{}\n".format(new_filename, *line[1:]))
#        cv2.imwrite(new_filename, img)
#    
#fw.close()

### Define Datasets

In [151]:
import random
import csv

TEST_SPLIT_RATIO = 0.2
VALIDATION_SPLIT_RATIO = 0.2

train_data = []
test_data = []
validation_data = []

with open("./data_cropped/driving_log_updated.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        rand = random.random()
        if rand < TEST_SPLIT_RATIO:
            test_data.append(line)
        elif rand < VALIDATION_SPLIT_RATIO+TEST_SPLIT_RATIO:
            validation_data.append(line)
        else:
            train_data.append(line)


In [152]:
total = 1.0* len(train_data)+len(test_data)+len(validation_data)
print(len(train_data), len(train_data)/total )
print(len(test_data), len(test_data)/total)
print(len(validation_data), len(validation_data)/total)


11473 0.6038103257723277
3808 0.2004105047102784
3720 0.19577916951739383


In [153]:
import sklearn
import random

def generator(samples, batch_size=32):
    num_samples = len(samples)
    samples = sklearn.utils.shuffle(samples)
    offset = 0
    while 1: # Loop forever so the generator never terminates
                    
        if offset >= len(samples):
            offset = 0
        vals = np.zeros((batch_size, 1))
        imgs = np.zeros((batch_size, 32, 32, 3))
        index = 0
        while index < batch_size:
            batch_sample = samples[offset]
            offset += 1
            if offset >= len(samples):
                offset = 0
            if False and abs(float(batch_sample[1])) <.3 and random.random() < .4:
                continue
            filename = batch_sample[0]
            center_image = cv2.imread(filename)
            imgs[index] = center_image
            vals[index][0] = float(batch_sample[1])
            index += 1

        # trim image to only see section with road
        X_train = imgs
        y_train = vals
            
        yield X_train, y_train


### Define Hyperparameters

In [154]:
epochs = 200
batch_size = 64

train_data_size = len(train_data)
test_data_size = len(test_data)
validation_data_size = len(validation_data)

### Define Model and Train

In [155]:
train_generator = generator(train_data, batch_size=batch_size)
validation_generator = generator(validation_data, batch_size=batch_size)
test_generator = generator(test_data, batch_size=batch_size)

In [134]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.advanced_activations import ELU
import keras.regularizers as regularizers

model = Sequential()
model.add(Cropping2D(cropping=((10,2),(0,0)), input_shape=(32, 32, 3)))
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(20, 32, 3)))
model.add(Conv2D(16, (5,5)))
model.add(ELU())
model.add(Conv2D(32, (5,5)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Flatten())
model.add(Dropout(.2))
model.add(Dense(1024,kernel_regularizer=regularizers.l2(0.01)))
model.add(ELU())
model.add(Dense(512,kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01)))
model.add(ELU())
model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01)))
model.add(ELU())
model.add(Dense(1))


In [113]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.advanced_activations import ELU

model = Sequential()
model.add(Cropping2D(cropping=((10,2),(0,0)), input_shape=(32, 32, 3)))
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(20, 32, 3)))
model.add(Conv2D(16, (8, 8), strides=(4, 4), padding="same"))
model.add(ELU())
model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same"))
model.add(ELU())
model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
model.add(Flatten())
model.add(Dropout(.2))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(1))

In [135]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(lr=.0001), loss='mse', metrics=[ 'mse'])

In [156]:
from keras.callbacks import ModelCheckpoint, Callback
checkpoint_callback = ModelCheckpoint('model{epoch:02d}.h5')
history = model.fit_generator(train_generator, steps_per_epoch=train_data_size/batch_size, \
                   validation_data=validation_generator, validation_steps=validation_data_size/batch_size, \
                   epochs=epochs, max_q_size=1, callbacks=[checkpoint_callback], initial_epoch=120)

Epoch 121/200
180/179 [==============================] - 13s - loss: 0.0254 - mean_squared_error: 0.0233 - val_loss: 0.0266 - val_mean_squared_error: 0.0251
Epoch 122/200
180/179 [==============================] - 13s - loss: 0.0244 - mean_squared_error: 0.0232 - val_loss: 0.0264 - val_mean_squared_error: 0.0256
Epoch 123/200
180/179 [==============================] - 13s - loss: 0.0238 - mean_squared_error: 0.0231 - val_loss: 0.0255 - val_mean_squared_error: 0.0251
Epoch 124/200
180/179 [==============================] - 13s - loss: 0.0237 - mean_squared_error: 0.0233 - val_loss: 0.0254 - val_mean_squared_error: 0.0251
Epoch 125/200
180/179 [==============================] - 13s - loss: 0.0234 - mean_squared_error: 0.0232 - val_loss: 0.0252 - val_mean_squared_error: 0.0251
Epoch 126/200
180/179 [==============================] - 13s - loss: 0.0233 - mean_squared_error: 0.0232 - val_loss: 0.0252 - val_mean_squared_error: 0.0251
Epoch 127/200
180/179 [==============================] - 1

180/179 [==============================] - 13s - loss: 0.0232 - mean_squared_error: 0.0232 - val_loss: 0.0249 - val_mean_squared_error: 0.0249
Epoch 174/200
180/179 [==============================] - 13s - loss: 0.0232 - mean_squared_error: 0.0232 - val_loss: 0.0252 - val_mean_squared_error: 0.0252
Epoch 175/200
180/179 [==============================] - 13s - loss: 0.0232 - mean_squared_error: 0.0232 - val_loss: 0.0251 - val_mean_squared_error: 0.0251
Epoch 176/200
180/179 [==============================] - 13s - loss: 0.0231 - mean_squared_error: 0.0231 - val_loss: 0.0254 - val_mean_squared_error: 0.0254
Epoch 177/200
180/179 [==============================] - 13s - loss: 0.0232 - mean_squared_error: 0.0232 - val_loss: 0.0250 - val_mean_squared_error: 0.0250
Epoch 178/200
180/179 [==============================] - 13s - loss: 0.0233 - mean_squared_error: 0.0233 - val_loss: 0.0248 - val_mean_squared_error: 0.0248
Epoch 179/200
180/179 [==============================] - 13s - loss: 0.0

### Evaluate

In [145]:
metrics = model.evaluate_generator(test_generator, steps=test_data_size/batch_size)

In [146]:
for i in zip(model.metrics_names, metrics):
    print(i)

('loss', 0.023760044500231742)
('mean_squared_error', 0.02109863519668579)


In [129]:
for i in zip(model.metrics_names, metrics):
    print(i)

('loss', 0.015913592446595431)
('mean_squared_error', 0.015913489945232867)


In [157]:
model.save("model_25.h5")

In [ ]:
from keras.models import load_model

model = load_model("model_6.h5")